
California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [2]:
import pandas as pd

In [3]:
housing = pd.read_csv('cal_housing_clean.csv')

In [4]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [5]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [6]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [7]:
y_val = housing['medianHouseValue']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [14]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [15]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [18]:
import tensorflow as tf

In [19]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [20]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [21]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [22]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_7ed5qvm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd62b671da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [23]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp_7ed5qvm/model.ckpt.
INFO:tensorflow:loss = 404145440000.0, step = 1
INFO:tensorflow:global_step/sec: 557.024
INFO:tensorflow:loss = 634884850000.0, step = 101 (0.181 sec)
INFO:tensorflow:global_step/sec: 565.422
INFO:tensorflow:loss = 541859120000.0, step = 201 (0.181 sec)
INFO:tensorflow:global_step/sec: 481.433
INFO:tensorflow:loss = 316506500000.0, step = 301 (0.204 sec)
INFO:tensorflow:global_step/sec: 567.176
INFO:tensorflow:loss = 693216200000.0, step = 401 (0.178 sec)
INFO:tensorflow:global_step/sec: 654.537
INFO:tensorflow:loss = 288225800000.0, step = 501 (0.151 sec)
INFO:tensorflow:global_step/sec: 667.808
INFO:tensorflow:loss = 732739400000.0, step = 601 (0.150 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 658.61
INFO:tensorflow:loss = 20702056000.0, step = 7701 (0.153 sec)
INFO:tensorflow:global_step/sec: 580.865
INFO:tensorflow:loss = 49380545000.0, step = 7801 (0.174 sec)
INFO:tensorflow:global_step/sec: 520.099
INFO:tensorflow:loss = 177432820000.0, step = 7901 (0.190 sec)
INFO:tensorflow:global_step/sec: 479.846
INFO:tensorflow:loss = 49215130000.0, step = 8001 (0.210 sec)
INFO:tensorflow:global_step/sec: 576.791
INFO:tensorflow:loss = 29033253000.0, step = 8101 (0.172 sec)
INFO:tensorflow:global_step/sec: 622.695
INFO:tensorflow:loss = 125802150000.0, step = 8201 (0.163 sec)
INFO:tensorflow:global_step/sec: 645.949
INFO:tensorflow:loss = 69949720000.0, step = 8301 (0.152 sec)
INFO:tensorflow:global_step/sec: 566.232
INFO:tensorflow:loss = 104142900000.0, step = 8401 (0.177 sec)
INFO:tensorflow:global_step/sec: 627.309
INFO:tensorflow:loss = 66264797000.0, step = 8501 (0.158 sec)
INFO:tensorflow:global_step/sec: 667.42
INFO:tensorflow:loss = 36634972

INFO:tensorflow:global_step/sec: 695.448
INFO:tensorflow:loss = 140130420000.0, step = 15601 (0.144 sec)
INFO:tensorflow:global_step/sec: 703.743
INFO:tensorflow:loss = 207422870000.0, step = 15701 (0.147 sec)
INFO:tensorflow:global_step/sec: 669.457
INFO:tensorflow:loss = 114939160000.0, step = 15801 (0.145 sec)
INFO:tensorflow:global_step/sec: 698.911
INFO:tensorflow:loss = 103972520000.0, step = 15901 (0.142 sec)
INFO:tensorflow:global_step/sec: 707.026
INFO:tensorflow:loss = 128509040000.0, step = 16001 (0.145 sec)
INFO:tensorflow:global_step/sec: 621.192
INFO:tensorflow:loss = 118119970000.0, step = 16101 (0.158 sec)
INFO:tensorflow:global_step/sec: 633.49
INFO:tensorflow:loss = 48471810000.0, step = 16201 (0.158 sec)
INFO:tensorflow:global_step/sec: 628.982
INFO:tensorflow:loss = 157867340000.0, step = 16301 (0.160 sec)
INFO:tensorflow:global_step/sec: 668.969
INFO:tensorflow:loss = 65339600000.0, step = 16401 (0.149 sec)
INFO:tensorflow:global_step/sec: 680.8
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 685.803
INFO:tensorflow:loss = 108983615000.0, step = 23501 (0.145 sec)
INFO:tensorflow:global_step/sec: 698.51
INFO:tensorflow:loss = 111710240000.0, step = 23601 (0.144 sec)
INFO:tensorflow:global_step/sec: 684.4
INFO:tensorflow:loss = 125964120000.0, step = 23701 (0.145 sec)
INFO:tensorflow:global_step/sec: 695.979
INFO:tensorflow:loss = 59564550000.0, step = 23801 (0.144 sec)
INFO:tensorflow:global_step/sec: 701.338
INFO:tensorflow:loss = 150433530000.0, step = 23901 (0.142 sec)
INFO:tensorflow:global_step/sec: 650.79
INFO:tensorflow:loss = 64348537000.0, step = 24001 (0.154 sec)
INFO:tensorflow:global_step/sec: 675.95
INFO:tensorflow:loss = 166627600000.0, step = 24101 (0.147 sec)
INFO:tensorflow:global_step/sec: 649.287
INFO:tensorflow:loss = 98160330000.0, step = 24201 (0.154 sec)
INFO:tensorflow:global_step/sec: 661.586
INFO:tensorflow:loss = 122276350000.0, step = 24301 (0.151 sec)
INFO:tensorflow:global_step/sec: 683.379
INFO:tensorflow:loss =

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [24]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [25]:
pred_gen = model.predict(predict_input_func)

In [26]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp_7ed5qvm/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [27]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [28]:
from sklearn.metrics import mean_squared_error

In [29]:
mean_squared_error(y_test,final_preds)**0.5

96792.41210239378